# Cleaning and Geocoding

Here, I clean, aggregate, and merge the various dataset extractions from our database at the unique 6-digit postal code level. I also extend our dataset by adding geolocation, specifically:

*  Latitude and longitude per postal code 
*  Neighbourhood in which the postal code resides. 

In [16]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import sqlalchemy as sql
import zipfile
import urllib as rq
import geopandas as gpd #for geomapping analysis
import requests #making url requests 
from io import BytesIO, StringIO
import re
import utm #for geomapping
import json #for geomapping 

In [ ]:
#pandas.read_sql(sql, con, index_col=None, coerce_float=True, params=None, parse_dates=None, columns=None, chunksize=None)

### 1. Basic Merging and Cleaning 

In [17]:
#Uploading and Cleaning  

data1 = pd.read_csv("M:\PH-RPM\Elba\Data\Manifold_Dowloads\MANIFOLD_DOWNLOAD_1.csv")
data2 = pd.read_csv("M:\PH-RPM\Elba\Data\Manifold_Dowloads\MANIFOLD_DOWNLOAD_2.csv")
data3 = pd.read_csv("M:\PH-RPM\Elba\Data\Manifold_Dowloads\MANIFOLD_DOWNLOAD_3.csv")
data4 = pd.read_csv("M:\PH-RPM\Elba\Data\Manifold_Dowloads\MANIFOLD_DOWNLOAD_4.csv")
data5 = pd.read_csv("M:\PH-RPM\Elba\Data\Manifold_Dowloads\MANIFOLD_DOWNLOAD_5.csv")

#Renaming postal code to match what's on file
data2 = data2.rename(columns={'Postal Code': 'Postal_Code'})
data3 = data3.rename(columns={'Postal Code': 'Postal_Code'})
data4 = data4.rename(columns={'Postal Code': 'Postal_Code'})
data5 = data5.rename(columns={'Postal Code': 'Postal_Code'})

#Renaming data5 columns to differentiate from useless columns in other datasets
data5.head()
data5 = data5.rename(columns={'Postal Code': 'Postal_Code', '35 to 44 years-Count-PC': 'Tot_Priv_HH_Age_35-44',\
                    '45 to 54 years-Count-PC':'Tot_Priv_HH_Age_45-54',\
                    '55 to 64 years-Count-PC':'Tot_Priv_HH_Age_55-64',\
                    '65 to 74 years-Count-PC': 'Tot_Priv_HH_Age_65-74',\
                     'Under 25 years-Count-PC':'Tot_Priv_HH_Age_u_25',\
                     '25 to 34 years-Count-PC': 'Tot_Priv_HH_Age_25-34', \
                    '75 years and over-Count-PC': 'Tot_Priv_HH_Age_75_o'})
#MERGING 
result_key = data1.merge(data2, on='Postal_Code', how='left')
result_key_1= result_key.merge(data3, on='Postal_Code', how='left')
result_key_2= result_key_1.merge(data4, on='Postal_Code', how='left')
result_final= result_key_2.merge(data5, on='Postal_Code', how='left')
result_final.to_csv('M:\PH-RPM\\Elba\\Data\\MANIFOLD_MERGED.csv')

In [18]:
#Renaming Household Income Data
result_final = result_final.rename(columns={'Household with income $0 - $4,999-Count-PC': 'HH_Inc_1', \
                            'Household with income $5,000 - $9,999-Count-PC': 'HH_Inc_2',\
                                'Household with income $10,000 - $14,999-Count-PC': 'HH_Inc_3' , \
                                'Household with income $15,000 - $19,999-Count-PC': 'HH_Inc_4', \
                            'Household with income $20,000 - $29,999-Count-PC': 'HH_Inc_5',\
                              'Household with income $30,000 - $39,999-Count-PC': 'HH_Inc_6', \
                                'Household with income $40,000 - $49,999-Count-PC': 'HH_Inc_7', \
                            'Household with income $50,000 - $59,999-Count-PC': 'HH_Inc_8', \
                          'Household with income $60,000 - $79,999-Count-PC': 'HH_Inc_9',\
                            'Household with income $80,000 - $99,999-Count-PC': 'HH_Inc_10', \
                           'Household with income $100,000 - $124,999-Count-PC': 'HH_Inc_11',\
                            'Household with income $125,000 - $149,999-Count-PC' : 'HH_Inc_12',\
                           'Household with income $150,000 and over-Count-PC' : 'HH_Inc_13'})

In [1]:
#list(result_final) 

In [20]:
data = result_final[['Postal_Code','Population_Total','Pop_15_plus','Dwellings_Tot','ChildCount',\
'Median_HH_Income','Health_Care_Spending','Financial_Services','Gifts_Money',\
'Charity_Contributions','Retirement_Savings','HH_Inc_1', 'HH_Inc_2', 'HH_Inc_3', \
'HH_Inc_4', 'HH_Inc_5', 'HH_Inc_6', 'HH_Inc_7', 'HH_Inc_8', 'HH_Inc_9', 'HH_Inc_10',\
'HH_Inc_11', 'HH_Inc_12', 'HH_Inc_13', 'Median value of dwellings $', \
'Median monthly shelter costs for owned dwellings ($)',\
'Average family income ($)', 'Owned-Count-PC', 'Rented-Count-PC','Band housing-Count-PC_y',\
'Tot_Priv_HH_Age_35-44','Tot_Priv_HH_Age_45-54', 'Tot_Priv_HH_Age_55-64','Tot_Priv_HH_Age_65-74',\
'Tot_Priv_HH_Age_25-34','Tot_Priv_HH_Age_u_25','Tot_Priv_HH_Age_75_o']]
data.to_csv('M:\PH-RPM\\Elba\\Data\\MANIFOLD_MERGED_CLEANED.csv')

In [ ]:
# Importing Foundation data for 2016-2017 and aggregating by postal code
foundation = pd.read_excel("M:\PH-RPM\Elba\Data\PBI_Data_8-10-2018.xlsx")
foundation = foundation.rename(columns={'ConcatPC': 'Postal_Code'})
found = foundation[['2017','2016','Postal_Code']]
grouped = found.groupby('Postal_Code')
foundation_donations = grouped.agg(np.sum)

In [61]:
#Merging foundation and manifold data
data = data.merge(foundation_donations, on='Postal_Code', how='left')
data.tail()
data.to_csv('M:\PH-RPM\\Elba\\Data\\MANIFOLD_BC_ALL.csv')

In [2]:
#list(data)

###  2. Here, we use Google API key to add lat/long to each postal code 

In [25]:
#https://developers.google.com/maps/documentation/javascript/get-api-key#standard-auth#
%matplotlib inline
#given a location via string, return lat lng.
def gc(location):
    
    #query the google geocode api for lat lng
    data = requests.get("https://maps.googleapis.com/maps/api/geocode/json?address=" + location + "&key=" + "key").json()
    
    if len(data['results']) > 0:
        
        #if there's a result, return the first one.
        return [data['results'][0]['geometry']['location']['lat'], data['results'][0]['geometry']['location']['lng']]
    
    else:
        #if no result is found, return an empty list
        return [0,0]

In [27]:
#Iterate over all values in Postal Code column to assign lat long 
def get_latlon(row):
    #obtain the lat/long for a row in assigned column (postal code)
    tup = gc(row.iloc[0])
    return pd.Series(tup[:2])

data[['Lat','Long']] = data[['Postal_Code']].apply(get_latlon , axis=1)
data.to_csv('M:\PH-RPM\\Elba\\Data\\Geocoded_MANIFOLD_BC_ALL.csv')

###  3. Now, we want to assign a neighbourhood to each postal code. For that we use neighbourhood shapefiles for the City of Vancouver 


In [70]:
#PLACE POSTAL CODES WITHIN NEIGHBOURHOOD (so that we can link the geojson file)
#First, we want to convert our shapefile to a geojson file 
#https://gis.stackexchange.com/questions/190903/assign-a-point-to-polygon-using-pandas-and-shapely
import geopandas
import geopandas.tools
import utm
from shapely.geometry import Point
from json import dumps

#Import data (already clean)
data= pd.read_csv('M:\PH-RPM\\Elba\\Data\\Geocoded_MANIFOLD_ALL.csv')

#Function to pass lat long to UTM projection
def getUTMs(row):
    tup = utm.from_latlon(row.iloc[0],row.ix[1])
    return pd.Series(tup[:2])

data[['utm_lat','utm_long']] = data[['Lat','Long']].apply(getUTMs , axis=1)

#Create the geometry column for the coordinates
data["geometry"] = data.apply(lambda row: Point(row['utm_lat'], row['utm_long']), axis=1)

#Convert to a GeoDataFrame
data = geopandas.GeoDataFrame(data, geometry="geometry")
data.crs = {'init': 'epsg:26910'}

# Declare the coordinate system for the places GeoDataFrame
# GeoPandas doesn't do any transformations automatically when performing
# the spatial join. The layers are already in the same CRS (WGS84) so no
# transformation is needed.
#http://spatialreference.org/ref/epsg/nad83-bc-albers/
#https://stackoverflow.com/questions/42751748/using-python-to-project-lat-lon-geometry-to-utm 
#https://pypi.org/project/utm/
#https://stackoverflow.com/questions/30014684/pandas-apply-utm-function-to-dataframe-columns

# Load the countries polygons
neighbourhoods = geopandas.GeoDataFrame.from_file("C:\\Users\\egomez\\Desktop\\local_area_boundary.shp")
neighbourhoods = geopandas.GeoDataFrame.from_file("C:\\Users\\egomez\\Desktop\\local_area_boundary.geojson")
neighbourhoods.crs = {'init': 'epsg:26910'}
#neighbourhoods.head()
# Drop all columns except the name and polygon geometry
neighbourhoods = neighbourhoods[["MAPID", "geometry"]]

# Perform the spatial join
result = geopandas.tools.sjoin(data, neighbourhoods, how="right")

# Print the results..
#data = result_final.drop(['Rank_x', 'Score_x', 'Population Total_x', 'Accum Population_x', \
#'Population 15+ Total_x'], axis=1)

C:\Users\egomez\AppData\Local\Continuum\anaconda3\envs\Env_1\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [72]:
#Note that this only has around 13,000 observations (only the postal codes that fall within our neighbourhoods)
result.to_csv('M:\PH-RPM\\Elba\\Data\\MANIFOLD_GEOCODED.csv')